# MQTT Subscribe 測試程式

這個 notebook 用於測試 MQTT 訂閱訊息功能。


In [2]:
import paho.mqtt.client as mqtt
import time
import json
from datetime import datetime


In [3]:
# MQTT Broker 設定（與 lesson6_1.ipynb 相同）
MQTT_BROKER = "localhost"  # 或使用 "192.168.0.252" 如果是遠端
MQTT_PORT = 1883
MQTT_USERNAME = "pi"
MQTT_PASSWORD = "raspberry"
MQTT_CLIENT_ID = "test_subscriber"

print(f"MQTT Broker: {MQTT_BROKER}:{MQTT_PORT}")
print(f"Username: {MQTT_USERNAME}")


MQTT Broker: localhost:1883
Username: pi


In [4]:
# 建立 MQTT 客戶端（使用 VERSION2 API）
try:
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2, client_id=MQTT_CLIENT_ID)
    use_v2 = True
except AttributeError:
    # 如果版本較舊，使用舊的方式
    client = mqtt.Client(client_id=MQTT_CLIENT_ID)
    use_v2 = False

# 設定使用者名稱和密碼
if MQTT_USERNAME and MQTT_PASSWORD:
    client.username_pw_set(MQTT_USERNAME, MQTT_PASSWORD)

# 連線回調函數（根據 API 版本使用不同的簽名）
if use_v2:
    def on_connect(client, userdata, flags, reason_code, properties):
        if reason_code == 0:
            print("✅ 成功連接到 MQTT Broker!")
        else:
            print(f"❌ 連線失敗，錯誤代碼: {reason_code}")
else:
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("✅ 成功連接到 MQTT Broker!")
        else:
            print(f"❌ 連線失敗，錯誤代碼: {rc}")

# 訂閱回調函數
def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print(f"✅ 成功訂閱主題 (Message ID: {mid}, QoS: {granted_qos})")

# 接收訊息回調函數
def on_message(client, userdata, msg):
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"\n📨 [{timestamp}] 收到訊息:")
    print(f"   主題: {msg.topic}")
    print(f"   QoS: {msg.qos}")
    try:
        # 嘗試解析 JSON
        data = json.loads(msg.payload.decode())
        print(f"   內容 (JSON): {json.dumps(data, ensure_ascii=False, indent=2)}")
    except:
        # 如果不是 JSON，直接顯示文字
        print(f"   內容: {msg.payload.decode()}")

# 設定回調函數
client.on_connect = on_connect
client.on_subscribe = on_subscribe
client.on_message = on_message

# 如果已經連線，先斷開
try:
    client.loop_stop()
    client.disconnect()
except:
    pass

# 連接到 Broker
try:
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.loop_start()  # 在背景執行網路迴圈
    time.sleep(1)  # 等待連線建立
except Exception as e:
    print(f"❌ 連線錯誤: {e}")


✅ 成功連接到 MQTT Broker!


## 測試 1: 訂閱單一主題


In [5]:
# 訂閱單一主題
topic = "客廳/message"
qos = 1

result, mid = client.subscribe(topic, qos)
print(f"訂閱主題: {topic}")
print(f"QoS: {qos}")
print(f"等待接收訊息... (可以執行 lesson6_1.ipynb 的發布程式來測試)")
print("按 Ctrl+C 或執行下一個 cell 來停止訂閱")


訂閱主題: 客廳/message
QoS: 1
等待接收訊息... (可以執行 lesson6_1.ipynb 的發布程式來測試)
按 Ctrl+C 或執行下一個 cell 來停止訂閱
✅ 成功訂閱主題 (Message ID: 1, QoS: [ReasonCode(Suback, 'Granted QoS 1')])


## 測試 2: 訂閱多個主題


In [ ]:
# 訂閱多個主題
topics = [
    ("客廳/message", 1),
    ("客廳/sensor", 1),
    ("客廳/counter", 1)
]

for topic, qos in topics:
    result, mid = client.subscribe(topic, qos)
    print(f"✅ 已訂閱: {topic} (QoS: {qos})")

print("\n等待接收訊息... (可以執行 lesson6_1.ipynb 的發布程式來測試)")
print("按 Ctrl+C 或執行下一個 cell 來停止訂閱")


✅ 已訂閱: 客廳/message (QoS: 1)
✅ 已訂閱: 客廳/sensor (QoS: 1)
✅ 已訂閱: 客廳/counter (QoS: 1)

等待接收訊息... (可以執行 lesson6_1.ipynb 的發布程式來測試)
按 Ctrl+C 或執行下一個 cell 來停止訂閱
✅ 成功訂閱主題 (Message ID: 2, QoS: [ReasonCode(Suback, 'Granted QoS 1')])
✅ 成功訂閱主題 (Message ID: 3, QoS: [ReasonCode(Suback, 'Granted QoS 1')])


✅ 成功訂閱主題 (Message ID: 4, QoS: [ReasonCode(Suback, 'Granted QoS 1')])


## 測試 3: 使用萬用字元訂閱（多層級 #）


In [7]:
# 使用萬用字元 # 訂閱所有以 "客廳/" 開頭的主題
topic = "客廳/#"  # # 表示匹配所有子主題
qos = 1

result, mid = client.subscribe(topic, qos)
print(f"✅ 已訂閱: {topic}")
print("說明: # 是萬用字元，會訂閱所有以 '客廳/' 開頭的主題")
print("例如: 客廳/message, 客廳/sensor, 客廳/counter 等")
print("\n等待接收訊息...")
print("按 Ctrl+C 或執行下一個 cell 來停止訂閱")


✅ 已訂閱: 客廳/#
說明: # 是萬用字元，會訂閱所有以 '客廳/' 開頭的主題
例如: 客廳/message, 客廳/sensor, 客廳/counter 等

等待接收訊息...
按 Ctrl+C 或執行下一個 cell 來停止訂閱
✅ 成功訂閱主題 (Message ID: 5, QoS: [ReasonCode(Suback, 'Granted QoS 1')])


## 測試 4: 使用萬用字元訂閱（單層級 +）


In [ ]:
# 使用萬用字元 + 訂閱單層級主題
topic = "客廳/+"  # + 表示匹配單一層級
qos = 1

result, mid = client.subscribe(topic, qos)
print(f"✅ 已訂閱: {topic}")
print("說明: + 是單層級萬用字元，只匹配一層")
print("會匹配: 客廳/message, 客廳/sensor")
print("不會匹配: 客廳/room/temperature (多層級)")
print("\n等待接收訊息...")
print("按 Ctrl+C 或執行下一個 cell 來停止訂閱")


✅ 已訂閱: 客廳/+
說明: + 是單層級萬用字元，只匹配一層
會匹配: 客廳/message, 客廳/sensor
不會匹配: 客廳/room/temperature (多層級)

等待接收訊息...
按 Ctrl+C 或執行下一個 cell 來停止訂閱


✅ 成功訂閱主題 (Message ID: 6, QoS: [ReasonCode(Suback, 'Granted QoS 1')])


## 測試 5: 持續監聽訊息（可設定時間）


In [ ]:
# 訂閱主題並持續監聽指定時間
topic = "客廳/#"
qos = 1

result, mid = client.subscribe(topic, qos)
print(f"✅ 已訂閱: {topic}")
print("開始監聽訊息...")

# 監聽 30 秒（可以修改這個時間）
listen_duration = 30
print(f"將監聽 {listen_duration} 秒...")

start_time = time.time()
while time.time() - start_time < listen_duration:
    time.sleep(1)

print(f"\n⏰ {listen_duration} 秒監聽時間結束")


✅ 已訂閱: 客廳/#
開始監聽訊息...
將監聽 30 秒...
✅ 成功訂閱主題 (Message ID: 7, QoS: [ReasonCode(Suback, 'Granted QoS 1')])

📨 [11:55:54] 收到訊息:
   主題: 客廳/message
   QoS: 1
   內容: Hello MQTT! 這是測試訊息

📨 [11:55:54] 收到訊息:
   主題: 客廳/sensor
   QoS: 1
   內容 (JSON): {
  "timestamp": "2025-11-30 11:55:54",
  "temperature": 25.5,
  "humidity": 60.0,
  "status": "正常"
}

📨 [11:55:54] 收到訊息:
   主題: 客廳/counter
   QoS: 1
   內容: 訊息編號: 1, 時間: 11:55:54

📨 [11:55:55] 收到訊息:
   主題: 客廳/counter
   QoS: 1
   內容: 訊息編號: 2, 時間: 11:55:55

📨 [11:55:56] 收到訊息:
   主題: 客廳/counter
   QoS: 1
   內容: 訊息編號: 3, 時間: 11:55:56

📨 [11:55:57] 收到訊息:
   主題: 客廳/counter
   QoS: 1
   內容: 訊息編號: 4, 時間: 11:55:57

📨 [11:55:58] 收到訊息:
   主題: 客廳/counter
   QoS: 1
   內容: 訊息編號: 5, 時間: 11:55:58

⏰ 30 秒監聽時間結束



📨 [13:08:46] 收到訊息:
   主題: 客廳/message
   QoS: 0
   內容: 測試


## 取消訂閱


In [ ]:
# 取消訂閱特定主題
topic = "客廳/#"
result, mid = client.unsubscribe(topic)
print(f"✅ 已取消訂閱: {topic}")


## 關閉連線


In [ ]:
# 關閉連線
client.loop_stop()
client.disconnect()
print("🔌 已斷開 MQTT 連線")
